In [106]:
#BY DANIEL CRUZ
#These are the libraries that are really needed for in order to make this whole task
import databricks.koalas as ks
import pandas as pd
import numpy as np
import datetime
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from datetime import datetime
import collections as cl
import os

In [ ]:
#Here, we are starting spark functions
sc = SparkContext.getOrCreate();

In [ ]:
w1=pd.read_csv("weather01.txt", sep ='\s+')
w2=pd.read_csv("weather02.txt", sep ='\s+')
w3=pd.read_csv("weather03.txt", sep ='\s+')
w4=pd.read_csv("weather04.txt", sep ='\s+')
w5=pd.read_csv("weather05.txt", sep ='\s+')
w6=pd.read_csv("weather06.txt", sep ='\s+')

In [ ]:
'''Since I had already worked with this, I only want to check the columns
names of this variable'''
w1.columns

In [67]:
#Here, I rename those columns
w1=w1.rename(columns={'X':'Number', 'year':'Year', 'month':'Month', 'measure':'Measure'})

In [68]:
#We can see now, that they have already changed
w1.columns

Index(['Number', 'Year', 'Month', 'Measure', 'X1', 'X2'], dtype='object')

In [69]:
#Now, we proceed to merge all the files in a single one
weather=pd.concat([w1,w2,w3,w4,w5,w6], axis=1)

In [70]:
#We visualise it in order to see if everythig is just fine
weather.head(3)

,Number,Year,Month,Measure,X1,X2,X3,X4,X5,X6,...,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31
1,1,2014,12,Max.TemperatureF,64,42,51,43,42,45,...,44,47,46,59,50,52,52,41,30,30
2,2,2014,12,Mean.TemperatureF,52,38,44,37,34,42,...,39,45,44,52,44,45,46,36,26,25
3,3,2014,12,Min.TemperatureF,39,33,37,30,26,38,...,33,42,41,44,37,38,40,30,22,20


In [71]:
#We can see that there is a columns that is not useful for us and we proceed to delete it
del(weather['Number'])

In [72]:
#We visualise it in order to see if everythig is just fine again
weather.head(3)

,Year,Month,Measure,X1,X2,X3,X4,X5,X6,X7,...,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31
1,2014,12,Max.TemperatureF,64,42,51,43,42,45,38,...,44,47,46,59,50,52,52,41,30,30
2,2014,12,Mean.TemperatureF,52,38,44,37,34,42,30,...,39,45,44,52,44,45,46,36,26,25
3,2014,12,Min.TemperatureF,39,33,37,30,26,38,21,...,33,42,41,44,37,38,40,30,22,20


In [73]:
#We proceed to make the  make the melting and creating a new column named value
weather = pd.melt(weather, 
                  id_vars=['Year', 'Month', 'Measure'], 
                  var_name='Day');
#We delete missing values
weather.dropna(inplace=True)
#In order to rename columns, we have to reset the index
weather = weather.reset_index(drop=True)

In [74]:
#We check how is everyhting going
weather.head(3)

,Year,Month,Measure,Day,value
0,2014,12,Max.TemperatureF,X1,64
1,2014,12,Mean.TemperatureF,X1,52
2,2014,12,Min.TemperatureF,X1,39


In [75]:
#Now, we change the name of the column value to Value
weather=weather.rename(columns={'value':'Value'})

In [76]:
#We create a function in order that the date shows up like YYYY-MM-DD
def creating_date(row):
    return "%d-%02d-%02d" % (row['Year'], row['Month'], int(row['Day'][1:]))

#We create a new column named Date and inside of it we apply the function
weather['Date'] = weather.apply(creating_date,axis=1)
#We choose the colums we want to use
weather = weather [['Measure', 'Value', 'Date']]

In [77]:
#We check how is everyhting going
weather.head(3)

,Measure,Value,Date
0,Max.TemperatureF,64,2014-12-01
1,Mean.TemperatureF,52,2014-12-01
2,Min.TemperatureF,39,2014-12-01


In [78]:
#This deletes the missing values

weather = weather.replace('<NA>',np.NaN)
weather = weather.dropna()

In [79]:
#Here, we can see that those missing values are not anymore therew.
weather.tail()

,Measure,Value,Date
8617,Mean.Wind.SpeedMPH,7,2015-10-31
8618,Max.Gust.SpeedMPH,17,2015-10-31
8619,PrecipitationIn,0.00,2015-10-31
8620,CloudCover,4,2015-10-31
8621,WindDirDegrees,224,2015-10-31


In [80]:
#Here we restruct the set by using pivot
weather = weather.pivot(index='Date', columns= 'Measure', values='Value')

In [81]:
#We check how everything is going
weather.head(3)

Measure,CloudCover,Events,Max.Dew.PointF,Max.Gust.SpeedMPH,Max.Humidity,Max.Sea.Level.PressureIn,Max.TemperatureF,Max.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Humidity,...,Mean.VisibilityMiles,Mean.Wind.SpeedMPH,MeanDew.PointF,Min.DewpointF,Min.Humidity,Min.Sea.Level.PressureIn,Min.TemperatureF,Min.VisibilityMiles,PrecipitationIn,WindDirDegrees
Date,,,,,,,,,,,,,,,,,,,,,
2014-12-01,6,Rain,46,29,74,30.45,64,10,22,63,...,10,13,40,26,52,30.01,39,10,0.01,268
2014-12-02,7,Rain-Snow,40,29,92,30.71,42,10,24,72,...,8,15,27,17,51,30.4,33,2,0.10,62
2014-12-03,8,Rain,49,38,100,30.4,51,10,29,79,...,5,12,42,24,57,29.87,37,1,0.44,254


In [82]:
#We rest the index again
weather = weather.reset_index()
#Here, whis only will take in consideration the values in a range of 365 because of the days in a year
weather = weather.drop(365,axis=0)

In [83]:
#We visualize the column events 
weather['Events']

0           Rain
1      Rain-Snow
2           Rain
3           Rain
4           Rain
5           Rain
6            NaN
7           Snow
8           Rain
9           Rain
10     Rain-Snow
11          Snow
12           NaN
13           NaN
14          Rain
15          Rain
16          Rain
17          Snow
18           NaN
19           NaN
20          Snow
21          Rain
22          Rain
23      Fog-Rain
24          Rain
25           NaN
26           NaN
27          Rain
28           NaN
29           NaN
         ...    
335         Rain
336          NaN
337          NaN
338          NaN
339          NaN
340          NaN
341          NaN
342         Rain
343         Rain
344         Rain
345         Rain
346          NaN
347          NaN
348          NaN
349         Rain
350         Rain
351          NaN
352          NaN
353          NaN
354          NaN
355         Rain
356         Rain
357          NaN
358          NaN
359          NaN
360          NaN
361          NaN
362         Ra

In [84]:
#We substitute the mising values with others that are random by using the method "ffill"
weather['Events']=weather['Events'].fillna(method='ffill')

In [85]:
#We proceed to sort the values from the Date column
weather = weather.sort_values('Date',ascending= False)
#We reset the index as Date 
weather = weather.set_index('Date',drop=True)
#We convert the index in the datetime way
weather.index = pd.to_datetime(weather.index)

In [86]:
#Now, we check the whole set by knowing its info
weather.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 365 entries, 2015-11-30 to 2014-12-01
Data columns (total 22 columns):
CloudCover                   365 non-null object
Events                       365 non-null object
Max.Dew.PointF               365 non-null object
Max.Gust.SpeedMPH            359 non-null object
Max.Humidity                 365 non-null object
Max.Sea.Level.PressureIn     365 non-null object
Max.TemperatureF             365 non-null object
Max.VisibilityMiles          365 non-null object
Max.Wind.SpeedMPH            365 non-null object
Mean.Humidity                365 non-null object
Mean.Sea.Level.PressureIn    365 non-null object
Mean.TemperatureF            365 non-null object
Mean.VisibilityMiles         365 non-null object
Mean.Wind.SpeedMPH           365 non-null object
MeanDew.PointF               365 non-null object
Min.DewpointF                365 non-null object
Min.Humidity                 365 non-null object
Min.Sea.Level.PressureIn     365 non-null o

In [87]:
#Now we change the type of data of those columns
weather['Max.TemperatureF'] = weather['Max.TemperatureF'].astype(int)
weather['Min.TemperatureF'] = weather['Min.TemperatureF'].astype(int)
weather['Mean.TemperatureF'] = weather['Mean.TemperatureF'].astype(int)
weather['Max.Dew.PointF'] = weather['Max.Dew.PointF'].astype(int)
weather['MeanDew.PointF'] = weather['MeanDew.PointF'].astype(int)
weather['Min.DewpointF'] = weather['Min.DewpointF'].astype(int)

In [88]:
#Here, we are converting it to a csv file 
koala1 = ks.from_pandas(weather)

In [89]:
#Here, we can check what the weather is, it should show up like a DataFrame
type(koala1)

databricks.koalas.frame.DataFrame

In [90]:
#Now we check if everything is fine 
koala1.head(5)

,CloudCover,Events,Max.Dew.PointF,Max.Gust.SpeedMPH,Max.Humidity,Max.Sea.Level.PressureIn,Max.TemperatureF,Max.VisibilityMiles,Max.Wind.SpeedMPH,Mean.Humidity,...,Mean.VisibilityMiles,Mean.Wind.SpeedMPH,MeanDew.PointF,Min.DewpointF,Min.Humidity,Min.Sea.Level.PressureIn,Min.TemperatureF,Min.VisibilityMiles,PrecipitationIn,WindDirDegrees
Date,,,,,,,,,,,,,,,,,,,,,
2015-11-30,6,Rain,26,17,75,30.53,38,10,14,65,...,10,9,23,18,54,30.39,28,10,0.00,65
2015-11-29,4,Rain,33,20,79,30.42,44,10,16,58,...,10,10,23,15,36,30.19,32,10,0.00,326
2015-11-28,8,Rain,50,23,93,30.2,60,10,18,80,...,9,10,43,36,67,30.11,41,4,0.21,358
2015-11-27,7,Rain,52,32,100,30.63,64,10,26,78,...,9,14,49,47,56,30.15,48,5,0.00,209
2015-11-26,6,Rain,49,28,100,30.87,59,10,22,79,...,9,10,42,34,57,30.64,38,5,0.00,180


In [99]:
#Here, we are converting it to a csv file 
weather.to_csv('weathercsv_outputfile.csv')

In [100]:
#Here, we close our pyspark session or work
sc.stop()